<a href="https://colab.research.google.com/github/luisdiaz1997/DataScienceCourse/blob/master/Week8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch import optim
import tensorflow.keras as keras


In [0]:
def linear(x, w, b):
  z = np.matmul(x,w) + b
  return z

In [0]:
def start_params(features = 2, classes = 1):
  params = {'W': np.random.randn(features, classes), 'b': np.random.randn(1, classes)}
  return params

In [0]:
def loss_MSE(model, inputs, targets):
  error = model(inputs) - targets
  return np.mean(error **2)/2

In [0]:
def loss_EL(model, inputs, targets):
  M = inputs.shape[0]
  error = targets*np.log(model(inputs) + 1e-3) + (1-targets)*np.log( 1 - model(inputs) + 1e-3)

  return np.sum(-error)/M


In [0]:
def grad(model, inputs, targets):
  M = inputs.shape[0]
  error =(model(inputs) - targets)/M

  grad_dict = {'dW': np.dot(inputs.T, error),
               'db': np.sum(error, axis = 0, keepdims=  True)}
  return grad_dict

In [0]:
def sigmoid(z):
  return 1/(1 + np.exp(-z))

In [0]:
class LinearRegression():
  def __init__(self, features = 2, classes = 1):
    self.params = start_params(features = features, classes = classes)
    self.costs = []
  
  def __call__(self, inputs):
    return linear(inputs, self.params['W'], self.params['b'])

  def fit(self, inputs, targets, learning_rate = 1e-3, epochs= 10):
    for i in tqdm(range(epochs)):
      loss = loss_MSE(self, inputs, targets)
      self.costs.append(loss)
      grads = grad(self, inputs, targets)

      for key, values in self.params.items():
        self.params[key] -= learning_rate * grads['d'+key]

In [0]:
class LogisticRegression():
  def __init__(self, features = 2, classes = 1):
    self.params = start_params(features = features, classes= classes)
    self.costs = []
  
  def __call__(self, inputs):
    Z = linear(inputs, self.params['W'], self.params['b'])
    return sigmoid(Z)

  def fit(self, inputs, targets, learning_rate = 1e-3, epochs= 10):
    for i in tqdm(range(epochs)):
      loss = loss_EL(self, inputs, targets)
      self.costs.append(loss)
      grads = grad(self, inputs, targets)

      for key, values in self.params.items():
        self.params[key] -= learning_rate * grads['d'+key]

In [0]:
data_url = "https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv"

In [0]:
df = pd.read_csv(data_url, sep=",")
df.head()

In [0]:
plt.figure(1)
x = df.households.values.reshape(-1, 1)
y = df.total_rooms.values.reshape(-1,1)
plt.figure(figsize=(8, 6))
plt.scatter(x[::10], y[::10])
plt.xlabel('Number of Households')
plt.ylabel('Total Rooms');

In [0]:
linear_model = LinearRegression(features = 1, classes = 1)

In [0]:
linear_model.fit(x, y, learning_rate=1e-8, epochs = 1000)

In [0]:
plt.plot(linear_model.costs)

In [0]:
plt.figure(2, figsize = (10,8))

y_predict = linear_model(x)
plt.scatter(x, y )
plt.scatter(x[:, 0], y_predict)


In [0]:
from sklearn.datasets import make_blobs
n_samples = 1500
random_state = 170
X, y = make_blobs(n_samples=n_samples, random_state=random_state, centers = np.array([[-3, -3], [1, 1], [5, -2]]))

In [0]:
def one_hot(data):
  new_data = np.zeros((len(data), np.max(data) + 1))
  for i, k in enumerate(data):
    new_data[i, k] = 1
  return new_data

In [0]:
Y = one_hot(y)

In [0]:
plt.figure(3, figsize = (10,8))
plt.scatter(X[:, 0], X[:, 1], c = y)
plt.colorbar();

In [0]:
logistic_model = LogisticRegression(features = 2, classes = 3)

In [0]:
logistic_model.fit(X, Y, learning_rate = 0.1, epochs = 500)

In [0]:
plt.plot(logistic_model.costs)

In [0]:
plt.figure(4, figsize = (10,8))

y_predict = np.argmax(logistic_model(X), axis = 1)
plt.scatter(X[:, 0], X[:, 1], c = y_predict.reshape(-1))
plt.colorbar();

In [0]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train/255
x_test = x_test/255 

In [0]:
plt.figure(figsize= (8, 8))
k = 10
for k in range(16):
  plt.subplot(4, 4, k+1)
  plt.imshow(x_train[k], cmap = 'gray')
  plt.title(y_train[k]);
  plt.axis('off')

In [0]:
x_train.shape

In [0]:
def flatten(data):
  new_data = data.reshape(data.shape[0], -1)
  return new_data

In [0]:
flatten(x_train).shape

In [0]:
mnist_model = LogisticRegression(features = flatten(x_train).shape[1], classes = 10)

In [0]:
mnist_model.params['W'].shape

In [0]:
mnist_model.fit(flatten(x_train), one_hot(y_train), learning_rate= 9, epochs =10)

In [0]:
plt.plot(mnist_model.costs)

In [0]:
plt.figure(figsize= (10, 5))
k = 10
for k in range(10):
  plt.subplot(2, 5, k+1)
  plt.imshow(mnist_model.params['W'][:, k].reshape(28, 28), vmin = -1.5, vmax =1.5, cmap = 'gray_r')
  plt.title(k)
  plt.axis('off');

In [0]:
from skimage import io
image = io.imread('https://i.kym-cdn.com/photos/images/original/000/531/557/a88.jpg')
plt.figure(figsize = (10, 5))
plt.imshow(image)
plt.axis('off');